## Serial Dilution
Get familiar with science-jubilee functionality using this serial dilution example notebook! We'll mix colors using a pipette tool.

In [ ]:
# Import the tools we'll need!
from science_jubilee.Machine import Machine
from science_jubilee.tools.Pipette import Pipette

In [ ]:
# Connect to the machine
m = Machine(address='192.168.1.2')

In [ ]:
# m.home_all() # if you need to, uncomment this line to home the machine

### 1. Set up our tools
We'll use a pipette tool to move liquid and a lab automation deck to hold labware 

In [ ]:
# we can check our tool indices if we need to
m.configured_tools

In [ ]:
pipette_index = 3               # if needed, change this to match the tool number for the pipette on your machine
pipette_config = "P300_config"  # change this to P300_config for a P300 pipette, P1000_config for a P1000 pipette
pipette_name = "Pipette"        # No need to change this

pipette = Pipette.from_config(
                            index = pipette_index,
                            name = pipette_name,
                            config_file = pipette_config
                            )
m.load_tool(pipette)

# We use a lab automation deck to hold labware
deck = m.load_deck('POSE-calibrated-deck')

### 2. Load our labware!
We'll use a 6 well plate to hold our stocks, and a 96-well plate to mix colors. Now, you should add:
- a tiprack for your pipette into slot 0
- a trash for your pipette into slot 5
- a 6 well plate with water in well A1, and water mixed with some watercoler in well A2, loaded into slot 1
- a 96 well plate into slot 3

Note we use slots 1 & 3 for the stocks and samples to minimize the travel distance.

You can prep these materials before moving on, or run the cells without liquid to do a dry run.

In [ ]:
# You might have EITHER a P300 or P1000 pipette and associated tiprack on your machine.
# uncomment the relevent line for your machine!

# tiprack = m.load_labware('opentrons_96_tiprack_300ul', 0)   # Uncomment if you have a P300 pipette, otherwise delete!
# tiprack = m.load_labware('opentrons_96_tiprack_1000ul', 0)  # Uncomment if you have a P100 pipette, otherwise delete!
trash = m.load_labware('generic_petri_dish_100ml', 5)         # Petri dish in slot 5, used as pipette tip trash
stocks = m.load_labware('scienfocus_6_wellplate_15900ul', 1)  # 6 well plate in slot 1
samples = m.load_labware('falcon_96_wellplate_360ul_flat', 3) # 96 well plate in slot 3

# For convenience, we can directly access our stock wells
water = stocks["A1"]                                          # plain water in well A1
color = stocks["A2"]                                          # with mixed with watercolor in well A2


In [ ]:
pipette.add_tiprack(tiprack) # Associate our pipette with our tiprack
pipette.trash = trash[0]     # The petri dish is a single well, so we set the 'well' as the trash

Now physically install this labware in the machine! Double check you're adding ther correct labware to the correct slots, and oriented the right way.

In [ ]:
# Drop the bed down so we have room to load our labware
m.move_to(z=150)


In [ ]:
# Pick up the pipette
m.pickup_tool(pipette)

### 3. Set up the sample plate
To set up the sample plate, we want to put water in all wells. Then, we want to put our color water into the first column to transfer down the line. 

Since it takes a good bit of time to transfer all the required liquid for an entire 96-well plate, we'll use the first 3 rows and first 3 columns as an example

The following cells will:
- move water into all the relevant wells 
- move colored water into the first column

In [ ]:
# We'll only transfer liquid to the first 3 rows and columns, since filling up an entire 96-wel plate takes a while
row_subset = list(samples.row_data)[0:3]
column_subset = list(samples.column_data)[0:3]
print(row_subset)       # rows A-
print(column_subset)    # columns 1-C

In [ ]:
# This well will fill all the relevant wells with water!
pipette.pickup_tip()
fill_volume = 50                                                        # Put 50 uL of water in all wells

for row in row_subset:
    for column in column_subset:
         pipette.transfer(
            vol = fill_volume,                                           # move the specified volume...
            source_well = water.bottom(3),                               # from the water well...
            destination_well = samples.wells[f"{row}{column}"].top(-1),  # to the sample well...
            new_tip = 'never'                                            # and use the same tip for all transfers
        )

# note we had to manually pickup a tip to use the 'never' tip strategy

In [ ]:
# This cell will add 100 uL of color water to the first row
fill_volume = 100
pipette.transfer(
                vol = fill_volume,                                      # move the specified volume
                source_well = color,                                    # from the color water well
                destination_well = samples.column_data[1][0:3],         # to the first 3 wells in column 1
                new_tip = 'never',                                      # use the same tip
                mix_after = (100, 2)                                    # and mix after dispensing
                )

### Run Serial Dilution!
The following cell will transfer and mix samples down each row of the well plate

In [ ]:
fill_volume = 100
for row in row_subset:
    for column in range(1, len(column_subset)):
        pipette.transfer(
            vol = fill_volume,
            source_well = samples.wells[f"{row}{column}"],
            destination_well = samples.wells[f"{row}{column+1}"],
            mix_after = (50, 2),
            new_tip = 'never'
        )

## End of Serial Dilution! 

In [ ]:
# You can drop the bed down to remove your labware
m.move_to(z=150)